# BBM409 - Introduction to Machine Learning Laboratory
### Assignment 2: Diabetes Risk Prediction using Decision Tree
### Atakan Yüksel - 21627892
### Burak Özüesen - 21827761

For data manipulation:

In [ ]:
import pandas as pd
import numpy as np

For visualization:

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt

## Creating Utility Functions

### K-fold Cross Validation

K-fold cross validation with k=5 will be used to evaluate our model. Instead of using traditional train_test_split, using k-fold cross validation is a better way to see how the model will handle unseen data.

In [ ]:
def cross_validation(df): # k-fold cross validation with k=5

    indices = np.arange(df.shape[0])
    np.random.shuffle(indices)

    bucket_1 = []
    bucket_2 = []
    bucket_3 = []
    bucket_4 = []
    bucket_5 = []

    for index, item in enumerate(indices):
        if index % 5 == 0:
            bucket_1.append(item)
        elif index % 5 == 1:
            bucket_2.append(item)
        elif index % 5 == 2:
            bucket_3.append(item)
        elif index % 5 == 3:
            bucket_4.append(item)
        else:
            bucket_5.append(item)
    
    df_subset_1 = df.iloc[bucket_1,:]
    df_subset_2 = df.iloc[bucket_2,:]
    df_subset_3 = df.iloc[bucket_3,:]
    df_subset_4 = df.iloc[bucket_4,:]
    df_subset_5 = df.iloc[bucket_5,:]

    return [df_subset_1, df_subset_2, df_subset_3, df_subset_4, df_subset_5]

### Discretization

We will apply discretization on "Age" column of our data. Decision Trees perform better on discrete data and discretization will be used on continuous columns. We will split "Age" into two bins. Higher bin counts cause an uneven distribution or lower accuracy.

In [ ]:
def discretization(data): # discretization on "Age" column of diabates_data. We are splitting into 2 bins. Higher bin numbers cause uneven distribution of "Age". Some bins end up with less than 5 examples while some bins have over 90.

    max_age = np.amax(data["Age"].to_numpy())
    min_age = np.amin(data["Age"].to_numpy())
    
    one_interval = (max_age - min_age) / 2
    
    first_bin = min_age + one_interval

    bins = [min_age, first_bin, max_age]
    temp = np.digitize(data["Age"], bins) - 1
    
    
    data["Age"] = temp.tolist()

### Entropy and Information Gain

There are different types of Decision Trees (difference come from feature selection techniques). Since we are implementing a ID3 decision tree, we will use entropy and information gain for feature selection.<br>
Entropy value of 0 means all samples belong to the same class, where entropy value of 1 means samples are distributed into classes equally.<br>
Information gain of 1 means all samples belong to the same class, where information gain of 0 means the class distribution has not changed.

In [ ]:
from math import log2

def entropy(pos, neg):
    try:
        return -((pos/(pos + neg)) * log2(pos/(pos + neg))) - ((neg/(pos + neg)) * log2(neg/(pos + neg)))
    except ValueError: # either pos or neg is 0, which means there is no entropy, all the remaining values belong to same 'class'.
        return 0
    except ZeroDivisionError: # there are no positive or negative examples, so by returning 1, we are telling the build_decision_tree() function to ignore this attribute.
        return 1

assert str(entropy(9, 5))[:5] == "0.940" # example from lecture slides.

In [ ]:
def information_gain(pos, neg, left_pos, left_neg, right_pos, right_neg):
    def helper(t_pos, t_neg, h_pos, h_neg):
        try:
            return (t_pos + t_neg) / (t_pos + t_neg + h_pos + h_neg)
        except ZeroDivisionError: # similar exception with entropy. there are no positive or negative examples so we are setting information gain to 0 to signal build_decision_tree() to ignore this attribute.
            return 0

    return entropy(pos, neg) - (helper(left_pos, left_neg, right_pos, right_neg) * entropy(left_pos, left_neg)) - (helper(right_pos, right_neg, left_pos, left_neg) * entropy(right_pos, right_neg))

assert str(information_gain(9, 5, 3, 4, 6, 1))[:5] == "0.151" # example from lecture slides.

## Pre-Processing

Reading data:

In [ ]:
diabetes_data = pd.read_csv("./data/diabetes_data_upload.csv")
diabetes_data.head(3)

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive


Applying discretization on continuous "Age" column:

In [ ]:
discretization(diabetes_data) # apply discretization on "Age" column.

Listing column data types:

In [ ]:
diabetes_data.dtypes # since all the answers are binary no need to have object datatypes.

Age                    int64
Gender                object
Polyuria              object
Polydipsia            object
sudden weight loss    object
weakness              object
Polyphagia            object
Genital thrush        object
visual blurring       object
Itching               object
Irritability          object
delayed healing       object
partial paresis       object
muscle stiffness      object
Alopecia              object
Obesity               object
class                 object
dtype: object

Converting all columns to category as they are binary:

In [ ]:
for column in diabetes_data.columns: # converting columns to category.
    diabetes_data[column] = diabetes_data[column].astype("category")
diabetes_data.dtypes

Age                   category
Gender                category
Polyuria              category
Polydipsia            category
sudden weight loss    category
weakness              category
Polyphagia            category
Genital thrush        category
visual blurring       category
Itching               category
Irritability          category
delayed healing       category
partial paresis       category
muscle stiffness      category
Alopecia              category
Obesity               category
class                 category
dtype: object

Converting all answers to 0 and 1:

In [ ]:
for column in diabetes_data.columns: # converting string answers to 0 and 1.
    if column != "Age":
        diabetes_data[column] = diabetes_data[column].replace(to_replace=["No", "Yes", "Negative", "Positive", "Male", "Female"], value=[0, 1, 0, 1, 0, 1])
diabetes_data.head(3)

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,0,0,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1


Gathering 5 folds for cross validation:

In [ ]:
data_subsets = cross_validation(diabetes_data) # 5-fold cross validation.

Implementing Node class as a building block for the decision tree:

In [ ]:
class Node: # this is a node in the decision tree. It stores it's name, it's neigbors(childs), what the prediction would be if decision tree stopped here, and the information gain achieved by this node.
    def __init__(self, name, neigbors, pred, information_gain):
        self._name = name
        self._neigbors = neigbors
        self._prediction = pred
        self._information_gain = information_gain

    def get_name(self):
        return self._name

    def set_name(self, name):
        self._name = name

    def get_neighbors(self):
        return self._neigbors

    def set_neighbors(self, neighbors): # this function is used to remove childs from a twig.
        self._neigbors = neighbors
    
    def add_neighbor(self, neighbor):
        self._neigbors.append(neighbor)

    def get_prediction(self):
        return self._prediction   

In [ ]:
def print_tree(node, level=0):
    try:
        leftChild = node.get_neighbors()[0]
        rightChild = node.get_neighbors()[1]
        print_tree(rightChild, level + 1)
        print(" " * 4 * level + "->", node.get_name())
        print_tree(leftChild, level + 1)
    except:
        pass

## Implementing the Model

The function build_decision_tree(df, root, target="class") takes 3 arguments. df is the training dataframe the model will train on, root is the starting Node of the decision tree, and target is the column name we are trying to predict. Function does not calculate the root on its own, so the root must be calculated beforehand and fed to the function.<br>Comments explain the flow.

In [ ]:
def build_decision_tree(df, root, target="class"):
    if len(df.columns) == 2: # only 'class' column is left
        return

    # since all the answers are binary we are building two branches where the answer is 0 and 1 respectively.

    # left branch
    left_branch = df[df[root.get_name()] == 0].copy(deep=True)
    left_branch.drop(columns=[root.get_name()], inplace=True)

    if left_branch.shape[0] == 0: # there are cases where some attributes are not used when there are no more rows left to process. In these cases we stop.
        return

    left_ig_dict = {} # this is used to store information gains for columns in str(column_name) : int(information_gain) format.
    for column in left_branch.columns:
        if column != target:
            neg_df = left_branch[left_branch[column] == 0] # the branch where the respective column is all 0s
            pos_df = left_branch[left_branch[column] == 1] # the branch where the respective column is all 1s

            left_ig_dict[column] = information_gain(len(left_branch[left_branch[target] == 1]), len(left_branch[left_branch[target] == 0]), len(neg_df[neg_df[target] == 1]), len(neg_df[neg_df[target] == 0]), len(pos_df[pos_df[target] == 1]), len(pos_df[pos_df[target] == 0]))
    
    left_index_max = np.argmax(list(left_ig_dict.values())) # since we store information gains in a dictionary, getting the maximum value is not straightforward.
    left_selected_feature = left_branch.columns[left_index_max]
    
    pred_index = np.argmax(left_branch[target].value_counts().to_dict().values()) # this where we know what the prediction would be if the decision tree stopped at this node. we are getting the counts of 0s and 1s and pick the prediction that occurs more.
    prediction = list(left_branch[target].value_counts().to_dict().keys())[pred_index]

    left_node = Node(left_selected_feature, [], prediction, left_ig_dict[left_selected_feature]) # constructing the node.
    root.add_neighbor(left_node) # adding a child.

    build_decision_tree(left_branch, left_node) # recursive call. by design the decision tree will try to build all left branches until it can't since we always call left branch before right branch in every recursive call.


    # same that happen with left branch happen here also. the difference is now the respective attribute is equal to 1.
    # right branch
    right_branch = df[df[root.get_name()] == 1].copy(deep=True)
    right_branch.drop(columns=[root.get_name()], inplace=True)

    if right_branch.shape[0] == 0:
        return

    right_ig_dict = {}
    for column in right_branch.columns:
        if column != target:
            neg_df = right_branch[right_branch[column] == 0]
            pos_df = right_branch[right_branch[column] == 1]

            right_ig_dict[column] = information_gain(len(right_branch[right_branch[target] == 1]), len(right_branch[right_branch[target] == 0]), len(neg_df[neg_df[target] == 1]), len(neg_df[neg_df[target] == 0]), len(pos_df[pos_df[target] == 1]), len(pos_df[pos_df[target] == 0]))
    
    right_index_max = np.argmax(list(right_ig_dict.values()))
    right_selected_feature = right_branch.columns[right_index_max]

    pred_index = np.argmax(right_branch[target].value_counts().to_dict().values())
    prediction = list(right_branch[target].value_counts().to_dict().keys())[pred_index]
    
    right_node = Node(right_selected_feature, [], prediction, right_ig_dict[right_selected_feature])
    root.add_neighbor(right_node)

    build_decision_tree(right_branch, right_node)

This is where we construct 5 decision trees for 5 different folds. As previously stated, root is being fed to the function.<br>Comments explain the flow.

In [ ]:
# Creating trees for each fold.

trees = []

for i in data_subsets: # data_subsets contain folds
    training_df = diabetes_data.drop(list(i.index.values)) # we remove folds from the main dataframe.

    target = "class"
    gains = {}

    for column in training_df.columns[:-1]: # not interested in 'class' column
        neg_df = training_df[training_df[column] == 0] # all negatives
        pos_df = training_df[training_df[column] == 1] # all positives

        gains[column] = information_gain(len(training_df[training_df[target] == 1]), len(training_df[training_df[target] == 0]), len(neg_df[neg_df[target] == 1]), len(neg_df[neg_df[target] == 0]), len(pos_df[pos_df[target] == 1]), len(pos_df[pos_df[target] == 0]))

    max_index = np.argmax(list(gains.values()))
    selection = training_df.columns[max_index] # we selected our root node.

    d_root = Node(selection, [], 0, 1) # creating the root.

    build_decision_tree(training_df, d_root) # time to create the tree with d_root as its root node.

    trees.append(d_root) # store the root in trees for classification metrics.

KeyboardInterrupt: 

This is where the prediction happens for each fold. We see the classification results for each fold.<br>Comments explain the flow.

In [ ]:
# this is where we see the classification results for the model. this is also where the prediction happens.
fig, axes = plt.subplots(1, 5, figsize=(25, 3))

misclassified_samples = [[], [], [], [], []]


for tree_index, data_subset in enumerate(data_subsets): # for every fold.

    test_df = data_subset
    training_tree = trees[tree_index]

    y_true = []
    y_pred = []

    for index, row in test_df.iterrows(): # for every row in the fold
        current_node = training_tree # root starting point
        while True:
            try:
                next_node = current_node.get_neighbors()[row[current_node.get_name()]] # row[current_node.get_name()] is either 0 and 1. if its 0 go to the left branch, it its 1 go to the right branch.
                current_node = next_node
            except: # we reached the end of the tree.
                y_true.append(row["class"])
                y_pred.append(current_node.get_prediction())
                if row["class"] != current_node.get_prediction():
                    misclassified_samples[tree_index].append(row)
                break

    print("Classification Metrics for Fold #{}".format(tree_index + 1))
    print(classification_report(y_true, y_pred) + "\n")

    cf_matrix = confusion_matrix(y_true, y_pred)

    group_names = ["True Neg","False Pos","False Neg","True Pos"]
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]

    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    axes[tree_index].set_title("Fold #{}".format(tree_index + 1))
    sns.heatmap(ax=axes[tree_index], data=cf_matrix, annot=labels, fmt="", cmap='Blues', vmin=0, vmax=65)

### Error Analysis for Classification

Fold #1 has the highest f1 scores for "Negative" and "Positive" class values, it also has highest precision for "Negative" and highest recall for "Positive"

Fold #2 #3 #4 are neither the best in any categories or the worst.

Fold #5 has the highest precision for "Positive" and highest recall for "Negative"

Since Fold #1 has the highest f1 scores, it is our best tree model.

Below is the visualization of Fold #1:

In [ ]:
print_tree(trees[0])

Here are some misclassified samples:

In [ ]:
for index, item in enumerate(misclassified_samples):
    print("Misclassified Sample for Fold #{}".format(index + 1))
    print(item[0], end="\n\n")

1st, 3rd, 4th samples have "partial paresis" == 1, which seems to be common among misclassified samples.
2nd misclassified sample could be an edge case.
5th misclassified sample only has "Gender" == 1, other features are 0.

## Decision Tree Pruning

Decision Trees tend to overfit a lot. There are multiple ways to tackle this problem like setting a maximum depth, ignoring information gains below a threshold, pruning etc. For this assignment, we will utilize pruning.

Splitting the data into 60-20-20 percents as train_df, test_df and validation_df respectively.

In [ ]:
def train_test_val_split(df): # train test val split with percentages 60-20-20 respectively.
    indices = np.arange(df.shape[0])
    np.random.shuffle(indices)

    train_bucket = []
    test_bucket = []
    validation_bucket = []

    for index, item in enumerate(indices):
        if index % 5 in [0,1,2]:
            train_bucket.append(item)
        elif index % 5 == 3:
            test_bucket.append(item)
        elif index % 5 == 4:
            validation_bucket.append(item)

    
    train_df = df.iloc[train_bucket,:]
    test_df = df.iloc[test_bucket,:]
    validation_df = df.iloc[validation_bucket,:]

    return train_df, test_df, validation_df

Training step is the same with previous step.

In [ ]:
train_df, test_df, validation_df = train_test_val_split(diabetes_data) # splitting the main dataframe

target = "class"
gains = {}

for column in train_df.columns[:-1]: # not interested in 'class' column
    neg_df = train_df[train_df[column] == 0] # only the rows where column = 0
    pos_df = train_df[train_df[column] == 1] # only the rows where column = 1

    gains[column] = information_gain(len(train_df[train_df[target] == 1]), len(train_df[train_df[target] == 0]), len(neg_df[neg_df[target] == 1]), len(neg_df[neg_df[target] == 0]), len(pos_df[pos_df[target] == 1]), len(pos_df[pos_df[target] == 0]))

max_index = np.argmax(list(gains.values()))
selection = train_df.columns[max_index] # selecting the feature with the highest information gain.

d_root = Node(selection, [], 0, 1)

build_decision_tree(train_df, d_root) # training the decision tree on train_df

Classification Metrics for the trained tree on test_df:

In [ ]:
# classification results on the test_df

y_true = []
y_pred = []

for index, row in test_df.iterrows(): 
    current_node = d_root
    while True:
        try:
            next_node = current_node.get_neighbors()[row[current_node.get_name()]]
            current_node = next_node
        except:
            y_true.append(row["class"])
            y_pred.append(current_node.get_prediction())
            break

print("Classification Metrics for 60-20-20 train-test-val split on test_df:")
print(classification_report(y_true, y_pred))

cf_matrix = confusion_matrix(y_true, y_pred)

group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(data=cf_matrix, annot=labels, fmt="", cmap='Blues')

Classification Metrics for the trained tree on validation_df before pruning:

In [ ]:
# classification results on the validation_df before pruning

y_true = []
y_pred = []

for index, row in validation_df.iterrows(): 
    current_node = d_root
    while True:
        try:
            next_node = current_node.get_neighbors()[row[current_node.get_name()]]
            current_node = next_node
        except:
            y_true.append(row["class"])
            y_pred.append(current_node.get_prediction())
            break

print("Classification Metrics for 60-20-20 train-test-val split on validation_df before pruning:")
print(classification_report(y_true, y_pred))

cf_matrix = confusion_matrix(y_true, y_pred)

group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(data=cf_matrix, annot=labels, fmt="", cmap='Blues')

Ruleset before pruning:

In [ ]:
print_tree(d_root)

Classification Metrics for the trained tree on validation_df after pruning:

In [ ]:
y_true = []
y_pred = []

for index, row in validation_df.iterrows(): 
    current_node = d_root # d_root is the root of the decision tree.
    while True:
        try:
            if len(current_node.get_neighbors()) == 1: # found a twig
                current_node.set_neighbors([]) # removing childs of the twig
                y_true.append(row["class"])
                y_pred.append(current_node.get_prediction())
                break
            next_node = current_node.get_neighbors()[row[current_node.get_name()]]
            current_node = next_node
        except: # reached the end of the tree
            y_true.append(row["class"])
            y_pred.append(current_node.get_prediction())
            break

print("Classification Metrics for 60-20-20 train-test-val split on validation_df after pruning:")
print(classification_report(y_true, y_pred))

cf_matrix = confusion_matrix(y_true, y_pred)

group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(data=cf_matrix, annot=labels, fmt="", cmap='Blues')

Ruleset after pruning:

In [ ]:
print_tree(d_root)

Since the initial accuracy is too high, pruning is ineffective for this dataset.